# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 5. Face Detection and Emotion Recognition (Курсовая работа)

Нужно написать приложение, которое будет считывать и выводить кадры с веб-камеры. В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре. После этого, человек показывает жесты руками, а алгоритм должен считать их и определенным образом реагировать на эти жесты.
На то, как система будет реагировать на определенные жесты - выбор за вами. Например, на определенный жест (жест пис), система будет здороваться с человеком. На другой, будет делать скриншот экрана. И т.д.
Для распознавания жестов, вам надо будет скачать датасет https://www.kaggle.com/gti-upm/leapgestrecog, разработать модель для обучения и обучить эту модель.

*(Усложненное задание) Все тоже самое, но воспользоваться этим датасетом:
https://fitnessallyapp.com/datasets/jester/v1

# Часть 3. Обучение и использование в онлайн-режиме

Работает следующим образом. После запуска алгоритма откроется cv2 окно. В этом окне нужно показать некий жест и затем нажать на клавиатуре цифру 0. Тем самым модель будет запоминать этот жест под классом 0. Затем показать второй жест и нажать на клавиатуре цифру 1. Тем самым модель будет запоминать второй жест под классом 1. И так нужно повторять в течение нескольких минут (примерно 2-3 минуты). Т.е. показать первый жест - нажать на 0, показать второй жест - нажать на 1. Эти операции можно чередовать по разному. Сначала можно запоминать жесты в центре кадра, а затем пробовать смещать влево-вправо.
Сверху экрана будет отображаться в первой строке предсказанный моделью класс (0 или 1). Во второй строке тензор с предсказаниями. В третьей строке значение ошибки. Нажатие на кнопку q - закрывает окно.

Пример работы с обученной моделью: https://youtu.be/jN3DwbXl8U8

In [1]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as tt
import matplotlib.pyplot as plt
import shutil
import math

from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as tt
import matplotlib.pyplot as plt

from PIL import Image

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Для обхода бага библиотек

In [4]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes, size_n, size_p, debug):
        super().__init__()
        
        self.debug = debug
        
        size_pre = in_channels
        size_cur = size_n
        size_c = size_p / 2
        self.conv1 = self.conv_block(size_pre, size_cur)
        self.conv2 = self.conv_block(size_cur, size_cur, pool=True)
        self.res1 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop1 = nn.Dropout(0.5)
        
        size_pre = size_cur
        size_cur = size_cur * 2
        size_c = size_c / 2
        self.conv3 = self.conv_block(size_pre, size_cur)
        self.conv4 = self.conv_block(size_cur, size_cur, pool=True)
        self.res2 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop2 = nn.Dropout(0.5)
        
        size_pre = size_cur
        size_cur = size_cur * 2
        size_c = size_c / 2
        self.conv5 = self.conv_block(size_pre, size_cur)
        self.conv6 = self.conv_block(size_cur, size_cur, pool=True)
        self.res3 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop3 = nn.Dropout(0.5)

        size_k = 6
        size_f = int((math.floor((size_c - size_k) / size_k) + 1)**2 * size_cur)
        self.maxp = nn.MaxPool2d(size_k)
        self.flat = nn.Flatten()
        self.line = nn.Linear(size_f, num_classes)
        
    @staticmethod
    def conv_block(in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                  nn.BatchNorm2d(out_channels), 
                  nn.ELU(inplace=True)]
        if pool: layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
    

    def call_layer(self, data, layer):
        if self.debug == True:
            print(self.counter, data.shape)
            self.counter += 1
        return layer(data)
            
        
    def forward(self, xb):
        self.counter = 0
        
        out = self.call_layer(xb, self.conv1)
        out = self.call_layer(out, self.conv2)
        #out = self.call_layer(out, self.res1) + out
        out = self.call_layer(out, self.drop1)
        
        out = self.call_layer(out, self.conv3)
        out = self.call_layer(out, self.conv4)
        #out = self.call_layer(out, self.res2) + out
        out = self.call_layer(out, self.drop2)
        
        out = self.call_layer(out, self.conv5)
        out = self.call_layer(out, self.conv6)
        #out = self.call_layer(out, self.res3) + out
        out = self.call_layer(out, self.drop3)

        out = self.call_layer(out, self.maxp)
        out = self.call_layer(out, self.flat)
        out = self.call_layer(out, self.line)
        
        return out

In [5]:
# Класс детектирования и обработки жестов с веб-камеры 
class GestureDetector(object):

    def __init__(self):
        self.epoch = 1     
        self.channels = 1
        self.classes = 2 #10
        self.size_n = 512
        self.size_p = 128

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = ResNet(self.channels, self.classes, self.size_n, self.size_p, False).to(self.device)

        max_lr = 0.001
        weight_decay = 1e-4
        self.optimizer = torch.optim.Adam(self.model.parameters(), max_lr, weight_decay=weight_decay)
        self.criterion = nn.CrossEntropyLoss()
        

    # Функция рисования найденных параметров на кадре
    def _draw(self, frame, text0, text1, text2):
        try:
            # пишем на кадре какой жест распознан
            cv2.putText(frame, text0, (30, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, text1, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, text2, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        except:
            print('Something wrong im draw function!')

        return frame
    
    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):    

        # Трансформации
        transforms = tt.Compose([# Настройки для расширения датасета
                                 tt.Grayscale(num_output_channels=1), # Картинки чернобелые
                                 tt.Resize((self.size_p,self.size_p), interpolation=Image.NEAREST),
                                 tt.ToTensor(),
                                 tt.Normalize((0.1307,), (0.3081,)),
        ])                      # Приведение к тензору
        
        train_on = False
        label = 0
        last_loss = ''
        
        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
            ret, frame = cap.read()
            #try:
            
            if train_on == True:
                train_on = False
                running_loss = 0.0

                for i in range(self.epoch):
                    data = Variable(transforms(Image.fromarray(frame))).unsqueeze(0)
                    labels = Variable(torch.tensor([label]))
                    data = data.to(self.device)
                    labels = labels.to(self.device)

                    self.optimizer.zero_grad()
                    outputs = self.model(data)
                    #print(outputs, labels)
                    
                    loss = self.criterion(outputs, labels)
                    loss.backward()
                    self.optimizer.step()
                    data.detach().cpu()
                    labels.detach().cpu()

                    running_loss += loss.item()
                    
                last_loss = f'{running_loss/self.epoch}'
                

            with torch.no_grad():
                data = Variable(transforms(Image.fromarray(frame))).unsqueeze(0).to(self.device)
                outputs = self.model(data)
                last_test = f'{outputs[0]}'
                predict_text = f'predict: {outputs[0].argmax()}'
                data.detach().cpu()


            # Рисуем на кадре
            self._draw(frame, predict_text, last_test, last_loss)

            #except:
                #print('Something wrong im main cycle!')

            # Показываем кадр в окне, и называем его(окно) - 'Gesture Detection'
            cv2.imshow('Gesture Detection', frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            key = chr(cv2.waitKey(1) & 0xFF)
            if key == 'q':
                break
            elif key >= '0' and key <= '9': # Для выбора номера класса
                label = int(key)
                train_on = True
                
        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()
        
        
# Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
cap = cv2.VideoCapture(0)  
# Создаем объект нашего класса приложения
fcd = GestureDetector()
# Запускаем
fcd.run()